In [ ]:
import Pkg
Pkg.activate("..\\..\\juMLia")
import MLDatasets: Wine
using Plots, DataFrames, JSON3
wine = Wine()

In [ ]:
wine.dataframe

---

The Wine dataset contains a chemical analysis of many different wines grown from one of three grape cultivars. While the perceptron isn't equipped to classify data into one of three groups, we can cut down our data to only consider two cultivars.

---

In [ ]:
perceptrondata = subset(wine.dataframe, :Wine => x -> (x .== 1 .|| x .== 2)) # Only select cultivars 1 and 2
perceptrondata = perceptrondata[:, [:OD, :Proline, :Wine]]
perceptronfeatures = perceptrondata[:, [:OD, :Proline]]
perceptronlabels = perceptrondata[:, :Wine]

In [ ]:
wine1 = subset(perceptrondata, :Wine => x -> x .== 1) # Only select cultivar 1
wine2 = subset(perceptrondata, :Wine => x -> x .== 2)

In [ ]:
scatter(wine1[:, :OD], wine1[:, :Proline], mc="red")
scatter!(wine2[:, :OD], wine2[:, :Proline], mc="blue")

One thing you'll notice is you can't actually draw a straight line here perfectly separating the two categories. I.e., they're not linearly separable. The perceptron is only able to classify across linear separations. So we do a little data filtering to remove the inseparable data.

---

In [ ]:
wine1vec = Vector.(eachrow(wine1)) # Only select cultivar 1
wine2vec = Vector.(eachrow(wine2))
filter!(x -> !(x[1] <= 3 && x[2] < 800), wine1vec)
filter!(x -> !(x[1] > 2.5 && x[2] > 800), wine2vec)
scatter([x[1] for x in wine1vec], [y[2] for y in wine1vec], mc="red")
scatter!([x[1] for x in wine2vec], [y[2] for y in wine2vec], mc="blue")

Much better!

---

In [ ]:
trainfeatures = [x[1:2] for x in [wine1vec; wine2vec]]
# The perceptron trains on labels either +1 or -1
trainlabels = map(val -> (val == 1) ? -1 : 1, [x[3] for x in [wine1vec; wine2vec]])

In [ ]:
include("SingleNeuron.jl")

In [ ]:
perceptronmodel = SingleNeuron(2, :perceptron)

This SingleNeuron constructor automatically instantiates a two-dimensional perceptron: it can take in a two-dimensional vector and spit out a binary value (either -1 or 1) depending on its current weights and bias. It starts with weights and bias set to 0, so it will output 1 for any input.

In [ ]:
predict(perceptronmodel, trainfeatures)

In [ ]:
trainlabels

In [ ]:
plotneuron(perceptronmodel; leftbound=1, rightbound=4)
scatter!([x[1] for x in wine1vec], [y[2] for y in wine1vec], mc="red")
scatter!([x[1] for x in wine2vec], [y[2] for y in wine2vec], mc="blue")

We train the perceptron by defining a loss function--in the perceptron's case, just the euclidean norm of the difference between a set of predicted labels and actual training labels--and descending down the gradient of the loss function at a particular prediction. We do this iteratively over a set of features and associated labels.

In [ ]:
train!(perceptronmodel, trainfeatures, trainlabels, numepochs=5)
plotneuron(perceptronmodel; leftbound=1, rightbound=4)
scatter!([x[1] for x in wine1vec], [y[2] for y in wine1vec], mc="red")
scatter!([x[1] for x in wine2vec], [y[2] for y in wine2vec], mc="blue")

There we go! The perceptron can find a dividing line between the two categories.